In [1]:
#Installing libraries
!pip install -q sentence-transformers==2.2.2 transformers==4.38.2 torch faiss-cpu rank-bm25==0.2.2 evaluate==0.4.1 
!pip -q echo 'Y' | uninstall huggingface-hub  
!pip install huggingface-hub==0.25.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 94.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 81.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 112.6 MB/s eta 0:00:00
/bin/bash: line 1: uninstall: command not found
ERROR: unknown command "echo" - maybe you meant "search"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 10.2 MB/s eta 0:00:0000:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the pack

In [2]:
#importing libraries
import torch
import faiss
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,AutoModelForCausalLM
from datasets import load_dataset
import re
import gradio as gr


In [37]:
#Declaring names of the important filenames
DATASET = "AlicanKiraz0/All-CVE-Records-Training-Dataset"
SENTENCE_TRANSFORMER = "sentence-transformers/all-MiniLM-L6-v2"
MODEL = "google/flan-t5-large"

In [3]:
#Preprocessing dataset
dataset = load_dataset(
    DATASET,
    split='train'
)
print(dataset)

README.md: 0.00B [00:00, ?B/s]

all_cve_database.jsonl:   0%|          | 0.00/475M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/297441 [00:00<?, ? examples/s]

Dataset({
    features: ['System', 'User', 'Assistant'],
    num_rows: 297441
})


In [4]:
#Extract relevant text from dataset

def extract_text(example):
    assistant_text = example.get('Assistant',"")
    #Fallback: if empty, try user column 
    if not assistant_text:
        assistant_text = example.get("User","")
    return {"text":assistant_text}

#Map the extraction function
dataset = dataset.map(extract_text)


Map:   0%|          | 0/297441 [00:00<?, ? examples/s]

In [5]:
#Visualize the dataset
print(dataset[3]['text'])

## CVE-2018-8090 Vulnerability Details

### CVE Metadata
- **CVE ID**: CVE-2018-8090
- **State**: PUBLISHED
- **Published Date**: July 25, 2018 at 23:00 UTC
- **Last Updated**: August 05, 2024 at 06:46 UTC
- **Reserved Date**: March 13, 2018 at 00:00 UTC
- **Assigned By**: mitre

### Vulnerability Description
Quick Heal Total Security 64 bit 17.00 (QHTS64.exe), (QHTSFT64.exe) - Version 10.0.1.38; Quick Heal Total Security 32 bit 17.00 (QHTS32.exe), (QHTSFT32.exe) - Version 10.0.1.38; Quick Heal Internet Security 64 bit 17.00 (QHIS64.exe), (QHISFT64.exe) - Version 10.0.0.37; Quick Heal Internet Security 32 bit 17.00 (QHIS32.exe), (QHISFT32.exe) - Version 10.0.0.37; Quick Heal AntiVirus Pro 64 bit 17.00 (QHAV64.exe), (QHAVFT64.exe) - Version 10.0.0.37; and Quick Heal AntiVirus Pro 32 bit 17.00 (QHAV32.exe), (QHAVFT32.exe) - Version 10.0.0.37 allow DLL Hijacking because of Insecure Library Loading.

### Affected Products

**n/a - n/a**
Affected versions:
- n/a (Status: affected)

### Refe

In [6]:
#Data Cleaning

def text_clean(example):
    text = example['text']
    if text is None:
        return {"text":""}
    text = re.sub(r"\s+"," ",text) #normalize whitespace
    text = text.strip()
    return {"text":text}

dataset = dataset.map(text_clean)
print("Final dataset size:", len(dataset))


Map:   0%|          | 0/297441 [00:00<?, ? examples/s]

Final dataset size: 297441


In [7]:
#Visualizing the text field from the dataset created.
print(dataset[528]['text'])

## CVE-2002-0379 Vulnerability Details ### CVE Metadata - **CVE ID**: CVE-2002-0379 - **State**: PUBLISHED - **Published Date**: April 02, 2003 at 05:00 UTC - **Last Updated**: August 08, 2024 at 02:49 UTC - **Reserved Date**: May 17, 2002 at 00:00 UTC - **Assigned By**: mitre ### Vulnerability Description Buffer overflow in University of Washington imap server (uw-imapd) imap-2001 (imapd 2001.315) and imap-2001a (imapd 2001.315) with legacy RFC 1730 support, and imapd 2000.287 and earlier, allows remote authenticated users to execute arbitrary code via a long BODY request. ### Affected Products **n/a - n/a** Affected versions: - n/a (Status: affected) ### References 1. [ESA-20020607-013](http://www.linuxsecurity.com/advisories/other_advisory-2120.html) 2. [4713](http://www.securityfocus.com/bid/4713) 3. [20020510 wu-imap buffer overflow condition](http://marc.info/?l=bugtraq&m=102107222100529&w=2) 4. [](http://www.washington.edu/imap/buffer.html) 5. [RHSA-2002:092](http://www.redhat.c

In [8]:
#chunk long description text

def chunk_text(text,chunk_size=400,overlap=50):
    if not text or not isinstance(text,str):
        return []
    chunks = []
    #Extract CVE-ID
    cve_match = re.search(r"CVE-\d{4}-\d+",text)
    cve_id = cve_match.group(0) if cve_match else 'UNKNOWN-CVE'

    #Identify sections
    affected_pattern = re.search(r"(Affected versions:\s*-)(.*?)(\n |$)",text,re.IGNORECASE | re.DOTALL)
    vulnerability_pattern = re.search(r"(### Vulnerability Description)(.*?)(### |$)",text,re.IGNORECASE | re.DOTALL)
    
    if affected_pattern is None and vulnerability_pattern is None:
        main_section = text
        affected_section = ""
        vulnerability_section = ""
    elif vulnerability_pattern is None:
        main_section = text[:affected_pattern.start()].strip()
        affected_section = affected_pattern.group(0).strip()
        vulnerability_section = ""
    elif ("No description available" in vulnerability_pattern.group(0)) & (affected_pattern is None):
        main_section = text
        affected_section = ""
        vulnerability_section = ""
    elif affected_pattern is None:
        main_section = text[:vulnerability_pattern.start()].strip()
        affected_section = ""
        vulnerability_section = vulnerability_pattern.group(0).strip()
    else:
        main_section = text[:vulnerability_pattern.start()].strip()
        affected_section = affected_pattern.group(0).strip()
        vulnerability_section = vulnerability_pattern.group(0).strip()
        
    # Add main section in chunks
    if main_section:
        chunks.append({
            "cve_id":cve_id,
            "text": main_section
        })

    #Split affected versions in sub chunks
    if affected_section:
        words = affected_section.split()
        start=0
        while start < len(words):
            end = min(start + chunk_size, len(words))
            sub_chunk = " ".join(words[start:end])
            chunks.append({
                "cve_id": cve_id,
                "text": sub_chunk
            })
            start += chunk_size - overlap
    #Split references in sub chunks
    if vulnerability_section:
        chunks.append({
            "cve_id": cve_id,
            "text": vulnerability_section
        })
        

    return chunks

all_chunks = []   
for text in dataset['text']:
    chunks = chunk_text(text)
    all_chunks.extend(chunks)
print("Total chunks:", len(all_chunks))
documents = all_chunks

Total chunks: 864919


In [9]:
#BM25 setup
bm25_texts = [doc["text"] for doc in documents]
bm25_cv_ids = [doc["cve_id"] for doc in documents]
tokenized_docs = [text.split() for text in bm25_texts]
bm25 = BM25Okapi(tokenized_docs)

In [38]:
#Dense embeddings + Faiss
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#embedder = SentenceTransformer(SENTENCE_TRANSFORMER,device=device)
embedder = SentenceTransformer(SENTENCE_TRANSFORMER,device=device)
#Extract text and Meta data
dense_texts = [doc["text"] for doc in documents]
dense_cve_ids  = [doc["cve_id"] for doc in documents]
doc_embeddings = embedder.encode(dense_texts,device=device,show_progress_bar=True,batch_size=16,normalize_embeddings=True,convert_to_numpy=True)
dimension = doc_embeddings.shape[1]
#Performing indexing based on FLATIP (cosine similarity)
index = faiss.IndexFlatIP(dimension)
index.add(np.array(doc_embeddings))

Batches:   0%|          | 0/54058 [00:00<?, ?it/s]

In [39]:
#Hybrid Retrieval function
def hybrid_search(query,top_k):

    #Check if query contains cve_id that matches the one stored in VDB
    cve_match = re.search(r"CVE-\d{4}-\d+",query,re.IGNORECASE)
    if cve_match:
        cve_id = cve_match.group(0).upper()
        #Return all chunks with matching cve_ids
        exact_match = [chunks for chunks in documents if chunks['cve_id']==cve_id]
        #Return top k relevant chunk
        return exact_match[:top_k]
    
    #Sparse Search
    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query) #calculate relevant score of document in a corpus for searched query. Returns list of scores.
    bm25_ranked_indices = sorted(
        range(len(bm25_scores)),
        key=lambda i: bm25_scores[i],
        reverse=True
    )[:top_k]
    #rank dictionary for rrf
    bm25_rank_dict = {
        idx:rank+1 for rank,idx in enumerate(bm25_ranked_indices)
    }
    #Dense Search
    query_embedding = embedder.encode([query],convert_to_numpy=True,normalize_embeddings=True)
    #bge_query = "Represent this sentence for searching relevant passages: " + query
    #query_embedding = embedder.encode(
        #[bge_query],
        #convert_to_numpy=True,
        #normalize_embeddings=True
    #)
    dense_scores, dense_indices = index.search(query_embedding,top_k) #Faiss uses IP distance metrics (cosine similarity)
    dense_indices = dense_indices[0]
    #rank for rrf
    dense_rank_dict = {
        idx: rank + 1
        for rank, idx in enumerate(dense_indices)
    }

    #RRF technique
    k=60 #Smoothing factor to prevent high-ranked items from dominating.
    all_candidate_indices = set(bm25_rank_dict.keys()) | set(dense_rank_dict.keys())
    rrf_scores = {}
    for idx in all_candidate_indices:
        score=0.0
        if idx in bm25_rank_dict:
            score += 1.0 / (k + bm25_rank_dict[idx])
        if idx in dense_rank_dict:
            score+= 1.0 / (k+ dense_rank_dict[idx])
        rrf_scores[idx] = score
    
    #sort by fused score
    final_ranked = sorted(
        rrf_scores.items(),
        key = lambda x:x[1],
        reverse=True
    )[:top_k]

    #Build final results
    results = []
    for idx, score in final_ranked:
        results.append({
            "cve_id": dense_cve_ids[idx],
            "text": dense_texts[idx],
            "score": float(score)
        })
    
    return results
    

In [64]:
#RAG Setup

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

def generate_text(query, top_k=3):

    retrieved_result = hybrid_search(query,top_k)
    context_block = []
    for item in retrieved_result:
        context_block.append(
            f"CVE ID: {item['cve_id']}\n"
            f"{item['text']}"
        )

    context = "\n".join(context_block)
    print("Top {} Retrieved Context: \n{} ".format(top_k,context))
    print()

    prompt = f"""
    You are a cybersecurity expert.
    Context : {context}
    Question: {query}
    Answer: 
    """
    
    inputs = tokenizer(prompt,return_tensors='pt',truncation=True, max_length=4096)
    outputs = model.generate(**inputs,max_new_tokens = 650,temperature=0.4,do_sample=False)
    # Decode only the generated answer (not the full prompt)
    generated_tokens = outputs[0]

    return tokenizer.decode(
        generated_tokens,
        skip_special_tokens=True
    ).strip()

In [53]:
#Test System
TOP_K = 3
#query = "Provide a comprehensive technical analysis of CVE-2005-1353, including exploitation vectors, impact assessment, and remediation strategies."
query = "Why are insecure deserialization flaws dangerous in enterprise systems?"
decoded_answer= generate_text(query,TOP_K)
print("Generated Answer: {}".format(decoded_answer))

Top 3 Retrieved Context: 
CVE ID: CVE-2019-19230
### Vulnerability Description An unsafe deserialization vulnerability exists in CA Release Automation (Nolio) 6.6 with the DataManagement component that can allow a remote attacker to execute arbitrary code. ###
CVE ID: CVE-2020-20136
### Vulnerability Description QuantConnect Lean versions from 2.3.0.0 to 2.4.0.1 are affected by an insecure deserialization vulnerability due to insecure configuration of TypeNameHandling property in Json.NET library. ###
CVE ID: CVE-2024-37361
### Vulnerability Description The application deserializes untrusted data without sufficiently verifying that the resulting data will be valid. (CWE-502) Hitachi Vantara Pentaho Business Analytics Server versions before 10.2.0.0 and 9.3.0.9, including 8.3.x, deserialize untrusted JSON data without constraining the parser to approved classes and methods. When developers place no restrictions on "gadget chains," or series of instances and method invocations that can s

In [54]:
#Evaluate Retriever
def evaluate_retriever(eval_data,top_k=5):
    total_queries = len(eval_data)
    precision_scores = []
    recall_scores = []
    hit_scores = []

    for sample in eval_data:
        query = sample['query']
        relevant_cves = set([i.upper() for i in sample["relevant_cve_ids"]])

        retrieved_chunks = hybrid_search(query,top_k=top_k)
        retrieved_cves = [chunks["cve_id"].upper() for chunks in retrieved_chunks]

        #---Precision@k
        relevant_retrieved = [cve for cve in retrieved_cves if cve in relevant_cves]
        precision_at_k = len(relevant_retrieved)/top_k
        precision_scores.append(precision_at_k)

        #------Recall@k
        if len(relevant_cves)>0:
            recall_at_k = len(set(relevant_retrieved))/len(relevant_cves)
        else:
            recall_at_k=0
        recall_scores.append(recall_at_k)

        #----Hit@k
        hit = 1 if len(relevant_retrieved)>0 else 0
        hit_scores.append(hit)

        results = {
        "Precision@{}".format(top_k): np.mean(precision_scores),
        "Recall@{}".format(top_k): np.mean(recall_scores),
        "Hit@{}".format(top_k): np.mean(hit_scores)
        }
    
    return results
#list of queries to evaluate upon
evaluation_data = [
    {
        "query": "Provide details of CVE-2024-13044",
        "relevant_cve_ids": ["CVE-2024-13044"]
    },
    {
        "query": "Remote code execution vulnerability in Apache Struts",
        "relevant_cve_ids": ["CVE-2017-9791"]
    },
    {
        "query": "Provide details of CVE-2024-21516",
        "relevant_cve_ids": ["CVE-2024-21516"]
    },
    {
        "query": "Provide details of CVE-2019-8608",
        "relevant_cve_ids": ["CVE-2019-8608"]
    },
    {
        "query": "Why are insecure deserialization flaws dangerous in enterprise systems?",
        "relevant_cve_ids": ["CVE-2024-37361"]
    },
]

retriever_res_eval = evaluate_retriever(evaluation_data, top_k=3)
print(retriever_res_eval)

{'Precision@3': np.float64(0.7333333333333333), 'Recall@3': np.float64(1.0), 'Hit@3': np.float64(1.0)}


In [56]:
#Evaluate faithfullness and calcuate context overlap scores which measures how much of the generated answer is semantically similar to the retrieved context using embeddings.

def evaluate_faithfullness(eval_queries,top_k=2):
    faithfullness_scores = []
    for query in eval_queries:
        #retrieve context
        retrieved_chunks = hybrid_search(query,top_k)
        if not retrieved_chunks:
            continue
        context_text = "\n".join(
            [f"CVE_ID: {i['cve_id']}\n{i['text']}" for i in retrieved_chunks]
        )
        #generate answers
        answer = generate_text(query,top_k)
        print("Generated answer is : {} ".format(answer))
        #embed context and answers
        context_embedding = embedder.encode(
            [context_text],
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        answer_embedding = embedder.encode(
            [answer],
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        #cosine similarity
        sim = cosine_similarity(answer_embedding,context_embedding)[0][0]
        print(sim)
        faithfullness_scores.append(sim)

    return {
        "Average Faithfulness Score": float(np.mean(faithfullness_scores)),
        "Min Score": float(np.min(faithfullness_scores)),
        "Max Score": float(np.max(faithfullness_scores))
    }

queries = [
    "Provide detailed explanation of CVE-2024-21516",
    "Explain remote code execution in Apache Struts",
    "How does SQL injection occur in web applications?",
]
results = evaluate_faithfullness(queries, top_k=2)
print(results)

Top 2 Retrieved Context: 
CVE ID: CVE-2024-21516
## CVE-2024-21516 Vulnerability Details ### CVE Metadata - **CVE ID**: CVE-2024-21516 - **State**: PUBLISHED - **Published Date**: June 22, 2024 at 05:00 UTC - **Last Updated**: January 14, 2025 at 16:25 UTC - **Reserved Date**: December 22, 2023 at 12:33 UTC - **Assigned By**: snyk
CVE ID: CVE-2024-21516
Affected versions: - 4.0.0.0 (Status: affected) ### References 1. [](https://security.snyk.io/vuln/SNYK-PHP-OPENCARTOPENCART-7266576) 2. [](https://github.com/opencart/opencart/commit/c546199e8f100c1f3797a7a9d3cf4db1887399a2) ### Credits - Calum Hutton 

Generated answer is : CVE-2024-21516 is a vulnerability in the OpenCart payment system that allows attackers to take advantage of a weakness in the payment system. 
0.72999907
Top 2 Retrieved Context: 
CVE ID: CVE-2017-9791
### Vulnerability Description The Struts 1 plugin in Apache Struts 2.1.x and 2.3.x might allow remote code execution via a malicious field value passed in a raw mess

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated answer is : The Struts 1 plugin in Apache Struts 2.1.x and 2.3.x might allow remote code execution via a malicious field value passed in a raw message to the ActionMessage. 
0.7954886
Top 2 Retrieved Context: 
CVE ID: CVE-2008-0908
### Vulnerability Description SQL injection vulnerability in browse.asp in Schoolwires Academic Portal allows remote attackers to execute arbitrary SQL commands via the c parameter. NOTE: the provenance of this information is unknown; the details are obtained solely from third party information. ###
CVE ID: CVE-2020-23978
### Vulnerability Description SQL injection can occur in Soluzione Globale Ecommerce CMS v1 via the parameter " offerta.php" ### 



/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated answer is : via the c parameter 
0.22451602
{'Average Faithfulness Score': 0.5833345651626587, 'Min Score': 0.22451601922512054, 'Max Score': 0.7954885959625244}


In [65]:
#For Testing on a UI using gradio
iface = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    title="Enterprise Cybersecurity RAG Assistant"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7864
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://dfbbebb55201dcd313.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Top 3 Retrieved Context: 
CVE ID: CVE-2019-19230
### Vulnerability Description An unsafe deserialization vulnerability exists in CA Release Automation (Nolio) 6.6 with the DataManagement component that can allow a remote attacker to execute arbitrary code. ###
CVE ID: CVE-2020-20136
### Vulnerability Description QuantConnect Lean versions from 2.3.0.0 to 2.4.0.1 are affected by an insecure deserialization vulnerability due to insecure configuration of TypeNameHandling property in Json.NET library. ###
CVE ID: CVE-2024-37361
### Vulnerability Description The application deserializes untrusted data without sufficiently verifying that the resulting data will be valid. (CWE-502) Hitachi Vantara Pentaho Business Analytics Server versions before 10.2.0.0 and 9.3.0.9, including 8.3.x, deserialize untrusted JSON data without constraining the parser to approved classes and methods. When developers place no restrictions on "gadget chains," or series of instances and method invocations that can s

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
